# Rotation-invariant similarity in time series using bag-of-patterns representation

Implementation of Lin et. al. (2012), a histogram-based representation for time series data, similar to the “bag of words” approach - called "bag of patterns".

_Inputs:_
- _S: list of  time series as a numpy arrays_
- _n: size of the sliding window_
- _w: word length - also length of the extracted patterns_
- _a: alphabet size_

## Basic Functions

In [1]:
def make_sentence(letter_list): 
    """
    Converts a list of strings into a string.
    E.g.: ['a', 'b', 'c'] --> 'abc'
    """
    return ''.join(l for l in letter_list)

def motifs_table(S_discrete_list):
    """
    Given a list of discrete subsequences, store their starting position and their frequency
    """
    table = defaultdict(list)
    freq = defaultdict(int)
    
    for i, seq in enumerate(S_discrete_list):
        word = make_sentence(seq)
        table[word].append(i)
        freq[word] += 1
    
    return table, freq

def extract_patterns(S, n):
    """
    Given a time series S as a numpy array, extract all subsequences, of length n, with a stride of 1.
    Returns a numpy array, each row corresponding to a subsequence. 
    
    Uses an indexer matrix for faster computation time.
    """
    
    N = len(S)
    
    # k x n matrix with the correct indices to extract the subsequences
    window_indexer = np.array(
        np.expand_dims(np.arange(n), 0) + 
        np.expand_dims(np.arange(N-n), 0).T
    )
    
    return S[window_indexer]

def plot_bop(S, M_all):    
    """
    Simple plot function
    """
    
    # Extract important parameters from the arrays
        # Merge all keys from the dictionaries in M_all, and reset their values to 0
    merged_dict = {}
    for m in M_all:
        merged_dict.update(m)
    global_dictionary = {key: 0 for key in merged_dict} 
        # Get the max values of the time series
    max_len = 0
    max_val = 2
    min_val = -2
    for s, m in zip(S, M_all):
        s = zscore_norm(s)
        if len(s) > max_len:
            max_len = len(s)
        if max(s) > max_val:
            max_val = max(s)
        if min(s) < min_val:
            min_val = min(s)
        
    # Plot 
    # f, axes = plt.subplots(len(S), 2, figsize=(15, len(S)*2))
    f = plt.figure(figsize=(15, len(S)*2))
    gs = f.add_gridspec(len(S), 3)
    colors = sns.color_palette("hls", 8)
    
    #for i, (s, m, ax1, ax2) in enumerate(zip(S, M_all, axes.ravel()[::2], axes.ravel()[1::2])):
    for i, (s, m) in enumerate(zip(S, M_all)):
        # Normalized time series
        ax1 = f.add_subplot(gs[i, 0])
        ax1.plot(zscore_norm(s), c=colors[i%len(colors)])
        #ax1.set_xlim((0, max_len))
        ax1.set_xlim((0, len(s)))
        ax1.set_ylim((min_val, max_val))
        # Histogram
        hist = {**global_dictionary , **m} # Merge dict so every histogram has the same x-axis
        keys = np.sort(list(hist.keys())) # Sort keys by alphabetical order
        vals = [list(hist.values())[i]/max(list(hist.values())) for i in np.argsort(list(hist.keys()))]
        ax2 = f.add_subplot(gs[i, 1:])
        ax2.bar(keys, vals, color=colors[i%len(colors)])
        ax2.set_ylim((0, 1.1))
        ax2.tick_params(axis="x", rotation=90, labelsize=12)
        
    plt.tight_layout()
    plt.show()
    
    return

## SAX Representation

### Normalization

In [ ]:
def zscore_norm(S):
    """
    z-normalization of a unidimensional time series. 
    Returns (value-mean)/std.
    """    
    
    if np.std(S) < 0.01:
        return S-np.mean(S)
    
    return (S-np.mean(S))/np.std(S)

### PAA

In [ ]:
def paa(S_norm, w):
    """
    Piecewise Aggregate Approximation (PAA) of a unidimensional time series (Keogh et. al. (2001)).
    S_norm is a normalized time series as a numpy array.
    w is the number of segments.
    """
    
    n = len(S_norm)
    S_paa = np.zeros(w)
        
    if n % w == 0:
        # Each segment will automatically have the same number of points
        l = n // w # Segment length
        for i in range(w):
            S_paa[i] = np.mean(S_norm[i*l:(i+1)*l])
            
    else:
        # We have to make sure that each segment gets an equal number of points
        points_per_seg = np.zeros(w)
        i_last_val = None
        i_last_segment = None
        
        for i in range(n*w):
            i_segment = i // n
            i_val = i // w            
            S_paa[i_segment] += S_norm[i_val]
            
            # Check that everything is fine 
            # print(f'n={n}, w={w}, i={i}, i_segment={i_segment}, i_val={i_val}')
            
        S_paa /= n

    return S_paa

### Discretization

In [ ]:
def lookup_table(a):
    
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
    table = {
        2: np.array([-np.inf,  0.00]),
        3: np.array([-np.inf, -0.43072, 0.43072]),
        4: np.array([-np.inf, -0.67448, 0, 0.67448]),
        5: np.array([-np.inf, -0.84162, -0.25334, 0.25334, 0.84162]),
        6: np.array([-np.inf, -0.96742, -0.43072, 0, 0.43072, 0.96742]),
        7: np.array([-np.inf, -1.06757, -0.56594, -0.18001, 0.18001, 0.56594, 1.06757]),
        8: np.array([-np.inf, -1.15034, -0.67448, -0.31863, 0, 0.31863, 0.67448, 1.15034]),
        9: np.array([-np.inf, -1.22064, -0.76470, -0.43072, -0.13971, 0.13971, 0.43072, 0.76470, 1.22064]),
        10: np.array([-np.inf, -1.28155, -0.84162, -0.52440, -0.25334, 0, 0.25334, 0.52440, 0.84162, 1.28155])
    }
    
    return table[a], alphabet[:a+1]

def discretization(S_paa, a):
    
    breakpoints, alphabet = lookup_table(a)
    S_discrete = []
    
    for val in S_paa:
        S_discrete.append(alphabet[np.where(breakpoints<val)[0][-1]])
    
    return S_discrete, breakpoints

### Distance Function

In [ ]:
def distance_table(a):
    
    breakpoints, _ = lookup_table(a)
    
    table = np.zeros((a, a))
    
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            if abs(i-j) <= 1:
                table[i, j] = 0
            else:
                table[i, j] = breakpoints[max(i, j)] - breakpoints[min(i, j)+1]
    
    return table

def dist(S_discrete_1, S_discrete_2, table):
    
    distance = np.zeros(len(S_discrete_1))
    
    for i, (s1, s2) in enumerate(zip(S_discrete_1, S_discrete_2)):
        distance[i] = table[ord(s1)-ord('a'), ord(s2)-ord('a')]
        
        # For verification purposes
        # print(f"i = {i}, s1 = {s1}, s2 = {s2}, value = {table[ord(s1)-ord('a'), ord(s2)-ord('a')]}")
        
    return np.sqrt(np.sum(distance**2))

def min_dist(S_discrete_1, S_discrete_2, n, w, a):
    """
    Given two symbolic representations of time series S_discrete_1 and S_discrete_2 
    as lists of the same length w, computes their distance (Lin et. al. (2007)).
    """
    
    prefactor = np.sqrt(n/w)
    table = distance_table(a)
    
    distance = dist(S_discrete_1, S_discrete_2, table)
    
    return prefactor*distance

## BOP Representation

### BOP Algorithm

In [ ]:
def BOP(S, n, w, a, numerosity_reduction=True, plot=True):
    """
    Implementation of Lin et. al. (2012), a histogram-based representation for time
    series data, similar to the “bag of words” approach - called "bag of patterns"
    
        - S: list of  time series as a numpy arrays
        - n: size of the sliding window
        - w: word length - also length of the extracted patterns
        - a: alphabet size
    """
    
    S_norm_all = []
    S_paa_all = []
    S_discrete_all = []
    hash_table_all = []
    M_all = []
        
    for s in S:
        subsequences = extract_patterns(s, n)

        S_norm_list = []
        S_paa_list = []
        S_discrete_list = []

        for seq in subsequences:
            S_norm = zscore_norm(seq)
            S_paa = paa(S_norm, w)
            S_discrete, breakpoints = discretization(S_paa, a)

            # Option for numerosity reduction - recommended for slow moving time series
            if numerosity_reduction:
                # If we have a succession of the same word, we only count it once
                if len(S_discrete_list):
                    word = ''.join(l for l in S_discrete)
                    if word == S_discrete_list[-1]:
                        pass
                    else:
                        S_norm_list.append(S_norm)
                        S_paa_list.append(S_paa)
                        S_discrete_list.append(''.join(l for l in S_discrete)) 
                
                # If it is the first subsequence, add everything to the arrays
                else:
                    S_norm_list.append(S_norm)
                    S_paa_list.append(S_paa)
                    S_discrete_list.append(''.join(l for l in S_discrete)) 

            # No numerosity reduction
            else:
                S_norm_list.append(S_norm)
                S_paa_list.append(S_paa)
                S_discrete_list.append(''.join(l for l in S_discrete))

        S_norm = np.concatenate(S_norm_list, axis=None)
        S_paa = np.concatenate(S_paa_list, axis=None)
        S_discrete = np.concatenate(S_discrete_list, axis=None)

        hash_table, M = motifs_table(S_discrete)

        S_norm_all.append(S_norm)
        S_paa_all.append(S_paa)
        S_discrete_all.append(S_discrete)
        hash_table_all.append(hash_table)
        M_all.append(M)
        
    if plot:
        plot_bop(S, M_all)
        
    return S_norm_all, S_paa_all, S_discrete_all, hash_table_all, M_all

### Distance Metric

In [ ]:
def bop_distance(M_all, plot=True):
    """
    The distance is defined as the Euclidian distance between the histograms, i.e. the 
    Euclidian distance between the pattern's occurrence frequency. 
    """
    
    distance_matrix = np.zeros((len(M_all), len(M_all)))
    for i in range(distance_matrix.shape[0]):
        for j in range(i+1, distance_matrix.shape[1]):
            dist = 0
            for key in M_all[i].keys():
                dist += (M_all[i][key]-M_all[j][key])**2
            dist= round(np.sqrt(dist), 2)
            distance_matrix[j, i] += dist
            
    if plot:
        f, ax = plt.subplots(figsize=(8, 8))
        mask = np.zeros_like(distance_matrix)
        mask[np.triu_indices_from(mask)] = True
        sns.heatmap(distance_matrix, mask=mask, annot=True, fmt='.1f',
                    square=True, cbar_kws={'shrink': 0.5}, cmap='Blues', ax=ax)
        plt.show()
        
    return distance_matrix